In [66]:
from pathlib import Path               # For easy handling of file paths
from PyPDF2 import PdfReader           # To read and extract text from PDF files
import re                              # For regular expressions (pattern matching)
import pandas as pd   


#https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/#1introduction
import numpy as np
import pandas as pd
import re


import json
import glob
import spacy

#Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt

#spacy
import spacy
from nltk.corpus import stopwords

#vis
import pyLDAvis
import pyLDAvis.gensim
import pandas as pd
from bertopic import BERTopic

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [67]:
import os
import random
import numpy as np
import tensorflow as tf  # if you are using tensorflow

In [68]:
os.environ['PYTHONHASHSEED'] = '0'
random.seed(0)
np.random.seed(0)
tf.random.set_seed(0)

In [69]:

from pathlib import Path               # For easy handling of file paths
from PyPDF2 import PdfReader           # To read and extract text from PDF files
# Directory containing PDFs
pdf_dir = Path("/Users/agnesnamyalo/Desktop/RESEARCH/DATA/Artifact_files")


# Function to extract date and text
def extract_date_and_text(pdf_path):
    try:
        reader = PdfReader(str(pdf_path))  # Open the PDF using PyPDF2
        full_text = ""                     # Initialize empty string to hold full PDF text
        for page in reader.pages:          # Loop through all pages in the PDF
            page_text = page.extract_text()   # Extract text from page
            if page_text:
                full_text += page_text + "\n"   # Add page text to full text

        # Extract date (e.g., May 19, 1998)
        #Looking for a date format like May 19, 1998 using regex and stores it. If none is found, it uses "Unknown Date"
        date_match = re.search(r'([A-Z][a-z]+ \d{1,2}, \d{4})', full_text)
        date = date_match.group(0) if date_match else "Unknown Date"

        # Clean out headers
        #RemovING repeated headers like CONGRESSIONAL RECORD — HOUSE that clutter the PDF content.
        cleaned_text = re.sub(r'CONGRESSIONAL RECORD\s+—\s+HOUSE.*?\n', '', full_text, flags=re.DOTALL)

        #Return the result as a dictionary

        return {
            "filename": pdf_path.name,   # Just the file name (not full path)
            "date": date,     # Extracting the date
            "Text": cleaned_text.strip()  # Cleaned text
        }
    except Exception as e:
        return {
            "filename": pdf_path.name,
            "date": "Error",
            "Text": f"Error reading file: {e}"
        }



# Extract from all PDFs in folder
records = []
for pdf_file in pdf_dir.glob("*.pdf"):   # Loop through all PDF files in the folder
    result = extract_date_and_text(pdf_file)   # Runing the extraction function
    records.append(result)    # Save the result in a list



# Store in DataFrame
df = pd.DataFrame(records)

# Preview and optionally save
#df.head()
#df.to_csv("congress_pdf_texts.csv", index=False)


Illegal character in Name Object (b'/\x84\xf8\x88\r\xf77\xc7\x01\x8b\xf8\x96\x03\xf8\x96\xf7s\x15\xfc\x88\x06}O\x05\xf8\x88\x06\t\x0e\x94\xf8')
Illegal character in Name Object (b'/\x84\xf8\x88\r\xf77\xc7\x01\x8b\xf8\x96\x03\xf8\x96\xf7s\x15\xfc\x88\x06}O\x05\xf8\x88\x06\t\x0e\x94\xf8')
Illegal character in Name Object (b'/\x84\xf8\x88\r\xf77\xc7\x01\x8b\xf8\x96\x03\xf8\x96\xf7s\x15\xfc\x88\x06}O\x05\xf8\x88\x06\t\x0e\x94\xf8')
Illegal character in Name Object (b'/\x98\xa0\xbe\x8b\xa2\x08\x92\x89\x8f\x85\x1e\x7f\x8b\x89vy\x86\x08Y}L\x96Mx\x08j\x81Y\\\x8b')


In [70]:
df.head()

,filename,date,Text
0,CREC-2005-07-12-pt1-PgD739-2.pdf,"July 12, 2005",CONGRESSIONAL RECORD —DAILY DIGEST D739 July 1...
1,CREC-2018-11-13-pt1-PgD1164.pdf,"November 13, 2018",CONGRESSIONAL RECORD — DAILY DIGEST D1164 Nove...
2,CREC-2006-03-09-pt1-PgD217-2.pdf,"March 9, 2006",CONGRESSIONAL RECORD —DAILY DIGEST D217 March ...
3,CREC-2002-06-18-pt1-PgD636.pdf,"June 18, 2002",CONGRESSIONAL RECORD —DAILY DIGEST D636 June 1...
4,CREC-2014-03-25-pt1-PgD309-2.pdf,"March 25, 2014",CONGRESSIONAL RECORD — DAILY DIGEST D309 March...


In [71]:
# Load your STM keywords
#stm_words_df = pd.read_csv("news_stm_topic_keywords_with_weights.csv")
#stm_keywords = set(w.lower() for w in stm_words_df['keyword'])


# Load your STM keywords
stm_words_df = pd.read_csv("news_stm_topic_keywords_with_weights.csv")
stm_keywords = set(
    w.lower() for w in stm_words_df['keyword']
    if w.lower() not in {'a.m.', 'p.m.'}
)

In [72]:
print(stm_keywords)

{'plant', 'habitat', 'river', 'school', 'timber', 'u.s.', 'water', 'animal', 'eagle', 'state', 'federal', 'office', 'threaten', 'mountain', 'court', 'california', 'agency', 'otter', 'vote', 'farmer', 'people', 'republican', 'subc', 'bald', 'tree', 'turtle', 'case', 'bear', 'hunt', 'work', 'protect', 'congress', 'butterfly', 'environmental', 'clinton', 'list', 'kill', 'bird', 'administration', 'wildlife', 'time', 'oil', 'president', 'owl', 'industry', 'fire', 'condor', 'island', 'service', 'breed', 'law', 'house', 'land', 'wolf', 'salmon', 'forest', 'protection', 'county', 'plan', 'american', 'whale', 'log', 'marine', 'senate', 'make', 'park', 'year', 'bill', 'area', 'sea', 'grizzly', 'fish', 'dam', 'nest'}


In [73]:
def filter_lines_with_keywords(text, keywords):
    lines = text.split('\n')
    filtered = [line for line in lines if any(keyword in line.lower() for keyword in keywords)]
    return "\n".join(filtered)

In [74]:
# Apply to the DataFrame
df['filtered_lines'] = df['Text'].apply(lambda x: filter_lines_with_keywords(x, stm_keywords))

In [75]:
df.head()

,filename,date,Text,filtered_lines
0,CREC-2005-07-12-pt1-PgD739-2.pdf,"July 12, 2005",CONGRESSIONAL RECORD —DAILY DIGEST D739 July 1...,CONGRESSIONAL RECORD —DAILY DIGEST D739 July 1...
1,CREC-2018-11-13-pt1-PgD1164.pdf,"November 13, 2018",CONGRESSIONAL RECORD — DAILY DIGEST D1164 Nove...,CONGRESSIONAL RECORD — DAILY DIGEST D1164 Nove...
2,CREC-2006-03-09-pt1-PgD217-2.pdf,"March 9, 2006",CONGRESSIONAL RECORD —DAILY DIGEST D217 March ...,CONGRESSIONAL RECORD —DAILY DIGEST D217 March ...
3,CREC-2002-06-18-pt1-PgD636.pdf,"June 18, 2002",CONGRESSIONAL RECORD —DAILY DIGEST D636 June 1...,CONGRESSIONAL RECORD —DAILY DIGEST D636 June 1...
4,CREC-2014-03-25-pt1-PgD309-2.pdf,"March 25, 2014",CONGRESSIONAL RECORD — DAILY DIGEST D309 March...,CONGRESSIONAL RECORD — DAILY DIGEST D309 March...


In [76]:
print(stm_keywords)

{'plant', 'habitat', 'river', 'school', 'timber', 'u.s.', 'water', 'animal', 'eagle', 'state', 'federal', 'office', 'threaten', 'mountain', 'court', 'california', 'agency', 'otter', 'vote', 'farmer', 'people', 'republican', 'subc', 'bald', 'tree', 'turtle', 'case', 'bear', 'hunt', 'work', 'protect', 'congress', 'butterfly', 'environmental', 'clinton', 'list', 'kill', 'bird', 'administration', 'wildlife', 'time', 'oil', 'president', 'owl', 'industry', 'fire', 'condor', 'island', 'service', 'breed', 'law', 'house', 'land', 'wolf', 'salmon', 'forest', 'protection', 'county', 'plan', 'american', 'whale', 'log', 'marine', 'senate', 'make', 'park', 'year', 'bill', 'area', 'sea', 'grizzly', 'fish', 'dam', 'nest'}


In [77]:
import spacy

def lemmatize_remove_stopwords(texts):
    nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
    nlp.max_length = 2_000_000
    cleaned_texts = []
    for text in texts:
        doc = nlp(text)
        tokens = [token.lemma_ for token in doc if not token.is_stop]
        cleaned_texts.append(" ".join(tokens))
    return cleaned_texts

# Usage example:
df['cleaned_text'] = lemmatize_remove_stopwords(df['filtered_lines'].tolist())


In [78]:
df.head()

,filename,date,Text,filtered_lines,cleaned_text
0,CREC-2005-07-12-pt1-PgD739-2.pdf,"July 12, 2005",CONGRESSIONAL RECORD —DAILY DIGEST D739 July 1...,CONGRESSIONAL RECORD —DAILY DIGEST D739 July 1...,congressional RECORD — DAILY DIGEST D739 July ...
1,CREC-2018-11-13-pt1-PgD1164.pdf,"November 13, 2018",CONGRESSIONAL RECORD — DAILY DIGEST D1164 Nove...,CONGRESSIONAL RECORD — DAILY DIGEST D1164 Nove...,congressional record — DAILY DIGEST D1164 Nove...
2,CREC-2006-03-09-pt1-PgD217-2.pdf,"March 9, 2006",CONGRESSIONAL RECORD —DAILY DIGEST D217 March ...,CONGRESSIONAL RECORD —DAILY DIGEST D217 March ...,congressional RECORD — DAILY DIGEST D217 March...
3,CREC-2002-06-18-pt1-PgD636.pdf,"June 18, 2002",CONGRESSIONAL RECORD —DAILY DIGEST D636 June 1...,CONGRESSIONAL RECORD —DAILY DIGEST D636 June 1...,congressional RECORD — DAILY DIGEST d636 June ...
4,CREC-2014-03-25-pt1-PgD309-2.pdf,"March 25, 2014",CONGRESSIONAL RECORD — DAILY DIGEST D309 March...,CONGRESSIONAL RECORD — DAILY DIGEST D309 March...,congressional RECORD — DAILY DIGEST D309 March...


In [79]:
# Remove all newline characters from the 'filtered_lines' column and save to 'cleaned_text'
#df['cleaned_text'] = df['filtered_lines'].str.replace('\n', ' ', regex=False)

# Preview the DataFrame with the cleaned column
#print(df[['filtered_lines', 'cleaned_text']].head())

In [80]:
def clean_remove_newlines_lower(texts):  #lowering the text
    return texts.str.replace('\n', ' ', regex=False).str.lower() #removing sentence separators eg \n

df['cleaned_text'] = clean_remove_newlines_lower(df['filtered_lines'])

In [81]:
# Remove words with both letters and digits
df['cleaned_text'] = df['cleaned_text'].str.replace(r'\b\w*\d\w*\b', '', regex=True)

# Optional: Remove extra whitespace
df['cleaned_text'] = df['cleaned_text'].str.replace(r'\s+', ' ', regex=True).str.strip()

In [82]:
import re

def clean_text(text):
    # Remove patterns like E:\CR\FM\D25MR4.REC dsk7sptvn1prod
    text = re.sub(r'E:\\CR\\FM\\D\d{2}[A-Z]{2}\d\.REC\s+\w+', '', text)
    # Remove a.m., p.m., am, pm
    text = re.sub(r'\b(a\.m\.|p\.m\.|am|pm)\b', '', text, flags=re.IGNORECASE)
    # Remove weird artifacts like 'SH–216'
    text = re.sub(r'\bSH–216\b', '', text)
    # Remove non-alpha "words" (e.g., digits, single letters, dash-words)
    text = re.sub(r'\b[^a-zA-Z]+\b', ' ', text)
    # Remove any remaining fragments of form "record", "hear", "hearing", "committee", etc. (optional, for cleaner results)
    text = re.sub(r'\b(record|hear|hearing|committee|entitle|presentation|response|legislation|act|amend|title|estate|deceased|veteran|benefit|burial|claim|request|program|center|pilot|property|lease|taxpayer|individual|unemployment|bond|financing)\b', '', text, flags=re.IGNORECASE)
    # Remove multiple spaces/newlines
    text = re.sub(r'\s+', ' ', text)
    # Strip leading/trailing spaces
    text = text.strip()
    return text
# Apply the function to your DataFrame
df['cleaned_text'] = df['cleaned_text'].apply(clean_text)

In [83]:
import re

# Optional: Define a set of valid two-letter English words to preserve ('an', 'in', 'is', 'on', etc.)
valid_two_letter_words = set(['am', 'an', 'as', 'at', 'be', 'by', 'do', 'go', 'he', 'if', 'in', 'is', 'it', 'me', 'my', 'no', 'of', 'on', 'or', 'so', 'to', 'up', 'us', 'we'])

def clean_text(text):
    # Remove repetitive file/record headers and boilerplate codes
    text = re.sub(r'\b(E|SH|FM|REC|CR|D\d{2}[A-Z]{2}\d|dsk7sptvn1prod)\b', '', text)
    text = re.sub(r'E:\\CR\\FM\\D\d{2}[A-Z]{2}\d\.REC(\s+\w+)*', '', text)
    # Remove time marks (am/pm/a.m./p.m.)
    text = re.sub(r'\b(a\.m\.|p\.m\.|am|pm)\b', '', text, flags=re.IGNORECASE)
    # Remove digit-only "words" and punctuation noise
    text = re.sub(r'\b[\d\W]+\b', ' ', text)
    # Remove single letters (isolated)
    text = re.sub(r'\b[a-zA-Z]\b', '', text)
    # Remove two-letter words not in valid set
    text = ' '.join([word for word in text.split() if len(word) > 2 or word.lower() in valid_two_letter_words])
    # Remove any extra spaces
    text = re.sub(r'\s+', ' ', text)
    # Strip leading/trailing spaces
    text = text.strip()
    return text

# Apply to DataFrame
df['cleaned_text'] = df['cleaned_text'].apply(clean_text)

In [84]:
df.head()

,filename,date,Text,filtered_lines,cleaned_text
0,CREC-2005-07-12-pt1-PgD739-2.pdf,"July 12, 2005",CONGRESSIONAL RECORD —DAILY DIGEST D739 July 1...,CONGRESSIONAL RECORD —DAILY DIGEST D739 July 1...,congressional daily digest july new public law...
1,CREC-2018-11-13-pt1-PgD1164.pdf,"November 13, 2018",CONGRESSIONAL RECORD — DAILY DIGEST D1164 Nove...,CONGRESSIONAL RECORD — DAILY DIGEST D1164 Nove...,congressional daily digest november house of r...
2,CREC-2006-03-09-pt1-PgD217-2.pdf,"March 9, 2006",CONGRESSIONAL RECORD —DAILY DIGEST D217 March ...,CONGRESSIONAL RECORD —DAILY DIGEST D217 March ...,congressional daily digest march national repu...
3,CREC-2002-06-18-pt1-PgD636.pdf,"June 18, 2002",CONGRESSIONAL RECORD —DAILY DIGEST D636 June 1...,CONGRESSIONAL RECORD —DAILY DIGEST D636 June 1...,congressional daily digest june of the resolut...
4,CREC-2014-03-25-pt1-PgD309-2.pdf,"March 25, 2014",CONGRESSIONAL RECORD — DAILY DIGEST D309 March...,CONGRESSIONAL RECORD — DAILY DIGEST D309 March...,congressional daily digest march achieving cle...


In [85]:
#Cleaning text column

# Droping NaN or missing values
df= df[df['cleaned_text'].notna()]

# Dropping empty strings or strings with only spaces
df= df[df['cleaned_text'].str.strip() != ""]

# Dropping very short texts (less than 3 words, which confuse UMAP/HDBSCAN)
df= df[df['cleaned_text'].str.split().str.len() > 2]

# Convert to list of strings for BERTopic
texts = df['cleaned_text'].astype(str).tolist()

print(f"Number of documents for topic modeling: {len(texts)}")

Number of documents for topic modeling: 325


In [86]:
#Cleaning text column

# Droping NaN or missing values
df= df[df['cleaned_text'].notna()]

# Dropping empty strings or strings with only spaces
df= df[df['cleaned_text'].str.strip() != ""]

# Dropping very short texts (less than 3 words, which confuse UMAP/HDBSCAN)
df= df[df['cleaned_text'].str.split().str.len() > 2]

# Convert to list of strings for BERTopic
texts = df['cleaned_text'].astype(str).tolist()

print(f"Number of documents for topic modeling: {len(texts)}")

Number of documents for topic modeling: 325


In [87]:
#exporting the wildlife dataset to be used for STM
df.to_csv('Artifact_congress_for_STMM.csv', index=False)

In [88]:
import numpy as np
import random
from bertopic import BERTopic
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import CountVectorizer
from sentence_transformers import SentenceTransformer
import umap

# Set seeds for full reproducibility
np.random.seed(42)
random.seed(42)

# Configure UMAP with fixed random state and params
umap_model = umap.UMAP(random_state=42, n_neighbors=15, min_dist=0.1, metric='cosine')

# Encode documents using SentenceTransformer (texts is your list from df['cleaned_text'])
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = embedding_model.encode(texts, show_progress_bar=True)

# KMeans clustering with fixed random state
kmeans_model = KMeans(n_clusters=9, random_state=42)
predicted_topics = kmeans_model.fit_predict(embeddings)

# Add stop word removal through CountVectorizer
vectorizer_model = CountVectorizer(stop_words='english')

# Instantiate BERTopic with reproducible settings and custom vectorizer
topic_model = BERTopic(
    language="english",
    nr_topics=None,
    umap_model=umap_model,
    vectorizer_model=vectorizer_model,
    verbose=True
)

# Fit using precomputed clusters (offline mode)
topics, probs = topic_model.fit_transform(texts, embeddings=embeddings, y=predicted_topics)

# Print topic information
print(topic_model.get_topic_info())


Batches:   0%|          | 0/11 [00:00<?, ?it/s]

2025-11-13 11:43:47,069 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-11-13 11:43:47,490 - BERTopic - Dimensionality - Completed ✓
2025-11-13 11:43:47,490 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-11-13 11:43:47,504 - BERTopic - Cluster - Completed ✓
2025-11-13 11:43:47,508 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-11-13 11:43:47,658 - BERTopic - Representation - Completed ✓


   Topic  Count                                        Name  \
0      0     58           0_subcommittee_states_senate_june   
1      1     54        1_house_subcommittee_vote_department   
2      2     51                2_senate_amendment_vote_yeas   
3      3     47        3_subcommittee_states_united_federal   
4      4     35         4_subcommittee_march_federal_fiscal   
5      5     30       5_states_united_subcommittee_national   
6      6     27  6_subcommittee_department_defense_security   
7      7     23            7_subcommittee_march_year_fiscal   

                                      Representation  \
0  [subcommittee, states, senate, june, united, h...   
1  [house, subcommittee, vote, department, public...   
2  [senate, amendment, vote, yeas, fund, neutral,...   
3  [subcommittee, states, united, federal, house,...   
4  [subcommittee, march, federal, fiscal, year, n...   
5  [states, united, subcommittee, national, entit...   
6  [subcommittee, department, defense, s

In [89]:
import pandas as pd



# Getting keywords for each topic (except -1, but in your case, there is no -1 so this check is redundant)
topic_keywords = {}
for topic_num in topic_model.get_topic_info()['Topic']:
    keywords = topic_model.get_topic(topic_num)
    keywords_str = ', '.join([kw[0] for kw in keywords])
    topic_keywords[topic_num] = keywords_str

# 2. Create the mapping DataFrame (this will be the same length/order as your df)
df_mapped = pd.DataFrame({
    'filename': df['filename'].values,
    'date': df['date'].values,
    'topics': topics
})

# Mapping the keywords for each topic
df_mapped['keywords'] = df_mapped['topics'].map(topic_keywords)

# Checking the number of rows
print(df_mapped.shape)
assert df_mapped.shape[0] == df.shape[0]


df_mapped.head()
df_mapped.to_csv("congress_BERTopic_topics_and_keywords.csv", index=False)


(325, 4)


In [90]:
import pandas as pd
words = pd.read_csv("congress_BERTopic_topics_and_keywords.csv")
print(words)

                             filename                date  topics  \
0    CREC-2005-07-12-pt1-PgD739-2.pdf       July 12, 2005       4   
1     CREC-2018-11-13-pt1-PgD1164.pdf   November 13, 2018       3   
2    CREC-2006-03-09-pt1-PgD217-2.pdf       March 9, 2006       4   
3      CREC-2002-06-18-pt1-PgD636.pdf       June 18, 2002       1   
4    CREC-2014-03-25-pt1-PgD309-2.pdf      March 25, 2014       6   
..                                ...                 ...     ...   
320    CREC-2014-02-27-pt1-PgD191.pdf   February 27, 2014       1   
321  CREC-2002-06-21-pt1-PgD660-2.pdf       June 21, 2002       1   
322    CREC-2003-05-14-pt2-PgD526.pdf        May 14, 2003       3   
323    CREC-1996-09-17-pt1-PgD954.pdf  September 17, 1996       0   
324    CREC-1995-08-03-pt2-PgD982.pdf      August 3, 1995       3   

                                              keywords  
0    subcommittee, march, federal, fiscal, year, na...  
1    subcommittee, states, united, federal, house, ...  


In [91]:
df_mapped.shape

(325, 4)

In [92]:

# Get unique topic/keywords pairs
keywords_df = df_mapped[['topics', 'keywords']].drop_duplicates().sort_values('topics').reset_index(drop=True)

print(keywords_df)


   topics                                           keywords
0       0  subcommittee, states, senate, june, united, ho...
1       1  house, subcommittee, vote, department, public,...
2       2  senate, amendment, vote, yeas, fund, neutral, ...
3       3  subcommittee, states, united, federal, house, ...
4       4  subcommittee, march, federal, fiscal, year, na...
5       5  states, united, subcommittee, national, entitl...
6       6  subcommittee, department, defense, security, u...
7       7  subcommittee, march, year, fiscal, hold, heari...


In [93]:
keywords_df.to_csv("congress_bertopic_keywords.csv", index=False)

In [94]:
#counting the number of times a keyword appears

from collections import Counter

# Splitting keywords and flatten to one list
all_keywords = []
for kw_string in keywords_df['keywords']:
    # Removiing spaces and split by comma, or use str.split(', ')
    words = [kw.strip() for kw in kw_string.split(',')]
    all_keywords.extend(words)

# Count frequency of each keyword
keyword_counts = Counter(all_keywords)

# Convert to DataFrame for easy handling/saving
keyword_counts_df = pd.DataFrame(keyword_counts.items(), columns=['keyword', 'count']).sort_values('count', ascending=False)

#print(keyword_counts_df)

keyword_counts_df.to_csv('Bertopic_keyword_counts_for_congress.csv', index=False)


In [95]:
#Getting the weights of the keywords

topic_nums = [topic for topic in topic_model.get_topic_info()['Topic']]

# Gather all (topic, keyword, weight) rows
records = []
for topic_num in topic_nums:
    keywords_with_weights = topic_model.get_topic(topic_num)
    for word, weight in keywords_with_weights:
        records.append({'topic': topic_num, 'keyword': word, 'weight': weight})

# Create dataframe of keyword weights per topic
keyword_weights_df = pd.DataFrame(records)

# View results
print(keyword_weights_df.tail())

keyword_weights_df.to_csv('congress_bertopic_keywords_with_weights_01.csv', index=False)



    topic   keyword    weight
75      7  hearings  0.041160
76      7     april  0.039461
77      7  national  0.032943
78      7   rayburn  0.032658
79      7   federal  0.028712


STM

In [96]:
import pandas as pd
stm_words = pd.read_csv("congress_stm_topic_keywords_with_weights.csv")
#print(stm_words)

In [97]:
#counting keywords in stm

keyword_counts_stm = stm_words.groupby('keyword').size().reset_index(name='count')

#print(keyword_counts_stm)

keyword_counts_stm.to_csv('STM_keyword_counts_for_congress.csv', index=False)


In [98]:
STM_topics = pd.read_csv("congress_STM_topics_and_keywords.csv")
STM_topics.head(2)

,filename,date,Text,filtered_lines,cleaned_text,document,topic,gamma,keywords
0,CREC-2005-07-12-pt1-PgD739-2.pdf,"July 12, 2005",CONGRESSIONAL RECORD —DAILY DIGEST D739 July 1...,CONGRESSIONAL RECORD —DAILY DIGEST D739 July 1...,congressional daily digest july new public law...,1,3,0.977161,"subcommittee, united, states, entitled, security"
1,CREC-2018-11-13-pt1-PgD1164.pdf,"November 13, 2018",CONGRESSIONAL RECORD — DAILY DIGEST D1164 Nove...,CONGRESSIONAL RECORD — DAILY DIGEST D1164 Nove...,congressional daily digest november house of r...,2,1,0.838841,"states, united, senate, located, service"
